# Research Analyzer - Multi-Analyst News Research System

This notebook implements a LangGraph-based research system that creates a team of specialized news analysts to conduct research on any topic and synthesize their findings into a comprehensive report.

## Phase 1: Setup & Environment

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python python-dotenv pydantic

In [ ]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")
_set_env("OPENAI_API_KEY")

In [ ]:
_set_env("TAVILY_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

## Phase 2: Data Models

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class NewsAnalyst(BaseModel):
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst.")
    description: str = Field(description="Description of the analyst's focus and expertise.")
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

class AnalystTeam(BaseModel):
    analysts: List[NewsAnalyst] = Field(description="Team of news analysts.")

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Search query for news retrieval.")

## Phase 3: State Definitions

In [ ]:
import operator
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import MessagesState

class GenerateAnalystsState(TypedDict):
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[NewsAnalyst]

class AnalysisState(MessagesState):
    max_num_turns: int
    context: Annotated[list, operator.add]
    analyst: NewsAnalyst
    analysis: str
    sections: Annotated[list, operator.add]

class ResearchGraphState(TypedDict):
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[NewsAnalyst]
    sections: Annotated[list, operator.add]
    introduction: str
    content: str
    conclusion: str
    final_report: str

## Phase 4: Prompt Templates

In [ ]:
ANALYST_INSTRUCTIONS = """You are tasked with creating a team of specialized news analysts. Follow these instructions:
1. Review the topic: {topic}
2. Examine any editorial feedback: {human_analyst_feedback}
3. Determine the most important perspectives for comprehensive news analysis.
4. Pick the top {max_analysts} perspectives.
5. Assign one analyst to each perspective with relevant expertise."""

QUESTION_INSTRUCTIONS = """You are a news analyst conducting research on {topic}.
Your goal is to gather specific, actionable insights.
Your analytical focus: {goals}
Begin by introducing yourself, then pose your analytical questions.
When satisfied, conclude with: \"Analysis complete!\""""

SEARCH_INSTRUCTIONS = """Generate a search query for recent news and information.
Focus on the latest developments relevant to the conversation."""

ANSWER_INSTRUCTIONS = """You are a news information expert.
Analyst focus: {goals}
Answer using this context: {context}
Guidelines: Use only provided context, include specific data, cite sources [1], [2], etc."""

SECTION_WRITER_INSTRUCTIONS = """You are a news report writer.
Create a concise section based on analyst research.
Structure: ## {focus} (title), ### Key Findings, ### Analysis, ### Sources
Maximum 300 words. Use numbered sources."""

REPORT_WRITER_INSTRUCTIONS = """You are creating a comprehensive news report on: {topic}
Task: Review all analyst sections, identify key insights, synthesize into cohesive narrative.
Format: Use markdown, start with ## News Analysis, preserve citations, create ## Sources section.
Analyst sections: {context}"""

INTRODUCTION_INSTRUCTIONS = """Write a compelling introduction for an analysis report on {topic}.
Target 100 words. Use markdown. Create # title, then ## Introduction section.
Report sections: {formatted_str_sections}"""

CONCLUSION_INSTRUCTIONS = """Write a conclusion for an analysis report on {topic}.
Target 100 words. Use markdown. Use ## Conclusion header.
Report sections: {formatted_str_sections}"""

## Phase 5: Analyst Generation

Create the analyst generation nodes and subgraph.

In [ ]:
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

def create_analysts(state: GenerateAnalystsState):
    """Create news analysts based on the topic."""
    topic = state['topic']
    max_analysts = state['max_analysts']
    human_analyst_feedback = state.get('human_analyst_feedback', '')
    
    structured_llm = llm.with_structured_output(AnalystTeam)
    system_message = ANALYST_INSTRUCTIONS.format(
        topic=topic, human_analyst_feedback=human_analyst_feedback, max_analysts=max_analysts
    )
    analysts = structured_llm.invoke(
        [SystemMessage(content=system_message)] + [HumanMessage(content="Generate the analyst team.")]
    )
    return {"analysts": analysts.analysts}

def human_feedback(state: GenerateAnalystsState):
    """No-op node for interruption."""
    pass

def should_continue(state: GenerateAnalystsState):
    """Return the next node to execute."""
    if state.get('human_analyst_feedback', None):
        return "create_analysts"
    return END

print("✓ Analyst generation functions defined!")

In [ ]:
# Build analyst generation graph
builder = StateGraph(GenerateAnalystsState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)
builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", should_continue, ["create_analysts", END])

memory = MemorySaver()
analyst_graph = builder.compile(checkpointer=memory)

display(Image(analyst_graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
# Test analyst generation
max_analysts = 3
topic = "Artificial Intelligence Safety Regulations"
thread = {"configurable": {"thread_id": "1"}}

for event in analyst_graph.stream({"topic": topic, "max_analysts": max_analysts}, thread, stream_mode="values"):
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Role: {analyst.role}")
            print(f"Affiliation: {analyst.affiliation}")
            print("-" * 50)